In [13]:
# imports
import pandas
from nltk import word_tokenize
from gensim.models import Word2Vec

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import classification_report

In [14]:
twitter_df = pandas.read_csv('processed_cyberbullying_tweets.csv')
tweets = twitter_df['processed_tweet_text'].apply(str).tolist()

labels = twitter_df['cyberbullying_type']

tokenized_sentences = [word_tokenize(sentence) for sentence in tweets]

model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

def tweet_to_vector(tweet, model):
    vector_list = [model.wv[word] for word in tweet if word in model.wv]
    if len(vector_list) == 0:
        return np.zeros(model.vector_size)
    vector = np.mean(vector_list, axis=0)
    return vector

lr_model = LogisticRegression()


tweet_vectors = np.array([tweet_to_vector(tweet, model) for tweet in tokenized_sentences])


X_train, X_test, y_train, y_test = train_test_split(tweet_vectors, labels, test_size=0.2, random_state=42)


lr_model.fit(X_train, y_train)


y_pred = lr_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8519760981234931
              precision    recall  f1-score   support

           0       0.65      0.29      0.40      1624
           1       0.87      0.97      0.92      7915

    accuracy                           0.85      9539
   macro avg       0.76      0.63      0.66      9539
weighted avg       0.83      0.85      0.83      9539



/Users/jackoehling/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
